# Full System

### Imports and Global Variables

In [169]:
import os
import copy
import torch
import random
import json
import jsonlines
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

#from nltk import tokenize
from torch.optim import Adam
from tqdm.notebook import tqdm
from rank_bm25 import BM25Okapi
from torch.utils.data import Dataset, DataLoader
from pyserini.search.lucene import LuceneSearcher
from sklearn.metrics import recall_score, accuracy_score, precision_score
from transformers import BertTokenizerFast, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration

#import pygaggle
from pygaggle.pygaggle.rerank.base import Query, Text, hits_to_texts
from pygaggle.pygaggle.rerank.transformer import MonoT5, MonoBERT

from scifact.scifact.evaluate.lib import metrics
from scifact.scifact.evaluate.lib.data import GoldDataset, PredictedDataset
from scifact.scifact.verisci.inference.merge_predictions import merge_one

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:1


### Read Json

In [4]:
def read_json(path):
    docs = []
    with jsonlines.open(path) as file:
        for line in file.iter():
            docs.append(line)
            
    return docs

corpus = read_json('data/corpus.jsonl')
train_json = read_json('data/claims_train.jsonl')
valid_json = read_json('data/claims_dev.jsonl')
test_json  = read_json('data/claims_test.jsonl')

### Metrics

In [134]:
def recall_full(true, preds):
    '''
        Calculate Recall for Document Retrieval
            Recall = No. of relevant documents retrieved / No. of total relevant documents
    
        Parameters
        ----------
        true : List
            List of ids of true document
        
        preds: Set
            List of relevant id documents
    '''
    count = 0.0
    for _id_ in preds:
        if _id_ in true:
            count += 1
    
    if count > len(true): count = len(true)

    return count / len(true)

def precision_full(true, preds):
    '''
        Calculate Precision for Document Retrieval
            Precision = No. of relevant documents retrieved / No. of total documents retrieved
    
        Parameters
        ----------
        true : List
            List of ids of true document
        
        preds: Set
            List of relevant id documents
        
    '''
    count = 0.0
    for _id_ in preds:
        if _id_ in true:
            count += 1
    
    if count > len(true): count = len(true)
        
    return count / len(doc)

def f1_score_full(recall, precision):
    '''
        Calculate F1 Score for Document Retrieval
            F-Score = 2 * Precision * Recall / Precision + Recall
    
        Parameters
        ----------
        recall : Float
        
        precision: Float
    '''
    
    return (2 * precision * recall) / (precision + recall)

### Pyserini

In [6]:
def pyserini_search(claim, top_k, return_hits=False):
    docs = []
    searcher = LuceneSearcher('scifact_index')
    hits = searcher.search(claim, k=top_k)

    for i in range(len(hits)):
        docs.append(json.loads(hits[i].raw))
     
    if return_hits: return hits
    
    return docs

### Abstract

In [108]:
def abstract_retrieval(claim, model):
    hits  = pyserini_search(claim, top_k=100, return_hits=True)
    query = Query(claim)
    texts = hits_to_texts(hits)
    reranked = model.rerank(query, texts)

    docs_id = []
    for i in range(0, 3):
        docs_id.append({'id': reranked[i].metadata['docid']})

    docs = []
    for _id_ in docs_id:
        for doc in corpus:
            if _id_['id'] == str(doc['doc_id']):
                docs.append(doc)
    return docs

### Sentence

In [109]:
def sentence_selection(model, docs, tokenizer):
    dataset = {'inputs': [], 'doc_id': []}
    for doc in docs:
        for sentence in doc['abstract']:
            dataset['inputs'].append(f'Query: {claim} Document: {sentence} Relevant: ')
            dataset['doc_id'].append(doc['doc_id'])
    
    encoding = tokenizer(dataset['inputs'], truncation=True, padding=True, return_tensors='pt', max_length=512)

    input_ids      = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs[0], 1)
    
    sentences = np.array(dataset['inputs'])[np.array(preds, dtype=bool)]
    docs_ids = np.array(dataset['doc_id'])[np.array(preds, dtype=bool)]
    
    return sentences, docs_ids

### Label

In [117]:
def label_prediction(model, sentences, docs_ids, tokenizer):
    last_id = docs_ids[0]
    idx = 0
    dataset = {'inputs': [], 'doc_id': []}

    hypothesis = f'hypothesis: {claim} '
    for s, id_ in zip(sentences, docs_ids):
        if id_ == last_id:
            idx += 1
            hypothesis += f"sentence{idx}: {s.split('Relevant')[0]} "
        elif id_ != last_id:
            dataset['inputs'].append(hypothesis)
            dataset['doc_id'].append(last_id)

            idx = 0
            hypothesis = f"hypothesis: {claim} sentence{idx}: {s.split('Relevant')[0]} "
        last_id = id_

    dataset['inputs'].append(hypothesis)
    dataset['doc_id'].append(last_id)
    
    encoding = tokenizer(dataset['inputs'], truncation=True, padding=True, return_tensors='pt', max_length=512)

    input_ids      = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs[0], 1)
    
    return preds, dataset

### monoBERT

In [107]:
abstract_model =  MonoBERT()

tokenizer_sentence = BertTokenizerFast.from_pretrained('castorini/monobert-large-msmarco-finetune-only', disable_tqdm=False)
sentence_model = BertForSequenceClassification.from_pretrained('castorini/monobert-large-msmarco-finetune-only')
sentence_model.load_state_dict(torch.load('bert_sentence_selection.pth'))
sentence_model.eval()

tokenizer_label = BertTokenizerFast.from_pretrained('bert-large-cased', disable_tqdm=False)
label_model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=3)
label_model.load_state_dict(torch.load('bert_label_prediction.pth'))
label_model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [126]:
data_len = 0
all_docs = {'docs': [], 'pred': []}
all_sentences = {'sentences': [], 'doc_ids': [], 'pred': []}

for sample in valid_json:
    claim   = sample['claim']
    correct = sample['evidence']
    
    if len(list(correct.keys())) < 1: continue
    data_len += 1
    
    docs                = abstract_retrieval(claim, abstract_model)
    sentences, docs_ids = sentence_selection(sentence_model, docs, tokenizer_sentence)
    preds, dataset      = label_prediction(label_model, sentences, docs_ids, tokenizer_label)
    
    all_docs['docs'].append(docs)
    all_docs['pred'].append(dataset)

    all_sentences['sentences'].append(senteces)
    all_sentences['doc_ids'].append(docs_ids)
    all_sentences['pred'].append(preds)

In [161]:
def adjust_for_metrics(dataset, rationale_selection):
    rationales = [line['sentences'] line in rationale_selection]
    labels = [line['pred'] for line in rationale_selection]
    
    rationale_ids = [x['claim_id'] for x in rationales]
    label_ids = [x['claim_id'] for x in labels]
    
    res = [merge_one(rationale, label) for rationale, label in zip(rationales, labels)]
    
    return res

In [16]:
## SCIFACT METRICS
data = GoldDataset(corpus, valid_json)
preds = adjust_for_metrics(all_docs, all_sentences)
predictions = PredictedDataset(all_docs, preds)
res = metrics.compute_metrics(predictions)
res

,sentence_selection,sentence_label,abstract_label_only,abstract_rationalized
precision,30.11,16.46,24.61,23.05
recall,31.20,15.95,26.78,20.12
f1,30.64,16.20,25.65,21.48


### monoT5

In [170]:
abstract_model =  MonoT5()

tokenizer_sentence  = T5Tokenizer.from_pretrained('castorini/monot5-base-msmarco')
sentence_model = T5ForConditionalGeneration.from_pretrained('castorini/monot5-base-msmarco')
sentence_model.load_state_dict(torch.load('T5_sentence_selection.pth'))
sentence_model.eval()

tokenizer_label = T5Tokenizer.from_pretrained('t5-large')
label_model = T5ForConditionalGeneration.from_pretrained('t5-large')
label_model.load_state_dict(torch.load('T5_label_prediction.pth'))
label_model.eval()

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [171]:
data_len = 0
all_docs = {'docs': [], 'pred': []}
all_sentences = {'sentences': [], 'doc_ids': [], 'pred': []}

for sample in valid_json:
    claim   = sample['claim']
    correct = sample['evidence']
    
    if len(list(correct.keys())) < 1: continue
    data_len += 1
    
    docs                = abstract_retrieval(claim, abstract_model)
    sentences, docs_ids = sentence_selection(sentence_model, docs, tokenizer_sentence)
    preds, dataset      = label_prediction(label_model, sentences, docs_ids, tokenizer_label)
    
    all_docs['docs'].append(docs)
    all_docs['pred'].append(dataset)

    all_sentences['sentences'].append(senteces)
    all_sentences['doc_ids'].append(docs_ids)
    all_sentences['pred'].append(preds)

In [20]:
# SCIFACT METRICS
data = GoldDataset(corpus, valid_json)
preds = adjust_for_metrics(all_docs, all_sentences)
predictions = PredictedDataset(all_docs, preds)
res = metrics.compute_metrics(predictions)
res

,sentence_selection,sentence_label,abstract_label_only,abstract_rationalized
precision,40.12,20.12,40.15,30.01
recall,41.10,21.01,42.12,29.51
f1,41.09,20.55,41.11,29.81
